In [ ]:
from __future__ import division

In [2]:
import os
import random
import math
import datetime
import time

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
import numpy as np
import pandas as pd

In [5]:
import tensorflow as tf
from keras import backend as K
from keras import regularizers

Using TensorFlow backend.


In [6]:
from keras.layers import Dense
from keras.objectives import categorical_crossentropy
from keras.metrics import categorical_accuracy
from keras.regularizers import l2
from keras.layers import Conv2D, Flatten
from keras.layers.core import Reshape
from keras.layers.merge import Concatenate

In [7]:
%load_ext autoreload
%autoreload 1

%aimport training

In [8]:
print "Done Importing"

Done Importing


In [9]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

# Architectures

## Arch 1: Normal dense fully-connected neural network

In [22]:
# Create a graph to hold the model.
graph_dense = tf.Graph()

# Create model in the graph.
with graph_dense.as_default():

    # Keras layers can be called on TensorFlow tensors:
    board = tf.placeholder(tf.float32, shape=(None, 42), name='board')
    outcome = tf.placeholder(tf.float32, shape=(None, 3), name='outcome')

    # Fully connected layers
    
    x = Dense(2048,
              activation='relu',
              kernel_regularizer=regularizers.l2(0.1),
              bias_regularizer=regularizers.l2(0.1),
              kernel_initializer='random_uniform',
              bias_initializer='zeros')(board)
    
    x = Dense(1024,
              activation='relu',
              kernel_regularizer=regularizers.l2(0.1),
              bias_regularizer=regularizers.l2(0.1),
              kernel_initializer='random_uniform',
              bias_initializer='zeros')(x)
   
    x = Dense(512,
              activation='relu',
              kernel_regularizer=regularizers.l2(0.1),
              bias_regularizer=regularizers.l2(0.1),
              kernel_initializer='random_uniform',
              bias_initializer='zeros')(x)
    
    x = Dense(48,
              activation='relu',
              kernel_regularizer=regularizers.l2(0.1),
              bias_regularizer=regularizers.l2(0.1),
              kernel_initializer='random_uniform',
              bias_initializer='zeros')(x)

    # output layer with 10 units and a softmax activation
    preds = Dense(3, activation='softmax', name='preds')(x) 
        
    with tf.name_scope('evaluation') as scope:
        
        loss = tf.identity(tf.reduce_mean(categorical_crossentropy(outcome, preds)), name='loss')
        accuracy = tf.identity(tf.reduce_mean(categorical_accuracy(outcome, preds)), name='accuracy')
                                    
        holdout_summaries = tf.identity(tf.summary.merge([
            tf.summary.scalar('holdout_loss', loss),
            tf.summary.scalar('holdout_accuracy', accuracy)]),
            name='holdout_summaries')
        
        batch_summaries = tf.identity(tf.summary.merge([
            tf.summary.scalar('batch_loss', loss),
            tf.summary.scalar('batch_accuracy', accuracy)]),
            name='batch_summaries')       
    
    with tf.name_scope('training') as scope:
        train_step = tf.train.AdamOptimizer(learning_rate=0.01).minimize(loss, name='train_step')
    
    # Initialize all variables
    init_op = tf.global_variables_initializer()
    
    #all_summaries = tf.summary.merge_all()

In [11]:
# test Env

In [12]:
ds_test = (training.DataLoader(frac_train=0.80, frac_test=0.20)
           .add_dataset('gen-1-2017-10-22-133109')
           .load())

In [ ]:
training.train(graph_dense, './models/dense_{}'.format(datetime.datetime.now().strftime("%Y_%m_%d_%H%M%S")),
               ds_test,  batch_size=100, num_batches=3000)

Running ./models/dense_2017_11_04_120354
Batch        0 Hold-Out Accuracy: 0.5129 Loss: 2.3747 Time taken: 0.1s
Batch     1000 Hold-Out Accuracy: 0.5392 Loss: 0.7456 Time taken: 34.7s
Batch     2000 Hold-Out Accuracy: 0.5318 Loss: 0.7183 Time taken: 39.9s


In [20]:
for i in graph_dense.get_operations():
    print i.name

board
outcome
dense_1/random_uniform/shape
dense_1/random_uniform/min
dense_1/random_uniform/max
dense_1/random_uniform/RandomUniform
dense_1/random_uniform/sub
dense_1/random_uniform/mul
dense_1/random_uniform
dense_1/kernel
dense_1/kernel/Assign
dense_1/kernel/read
dense_1/Square
dense_1/mul/x
dense_1/mul
dense_1/Const
dense_1/Sum
dense_1/add/x
dense_1/add
dense_1/Const_1
dense_1/bias
dense_1/bias/Assign
dense_1/bias/read
dense_1/Square_1
dense_1/mul_1/x
dense_1/mul_1
dense_1/Const_2
dense_1/Sum_1
dense_1/add_1/x
dense_1/add_1
dense_1/MatMul
dense_1/BiasAdd
dense_1/Relu
dense_2/random_uniform/shape
dense_2/random_uniform/min
dense_2/random_uniform/max
dense_2/random_uniform/RandomUniform
dense_2/random_uniform/sub
dense_2/random_uniform/mul
dense_2/random_uniform
dense_2/kernel
dense_2/kernel/Assign
dense_2/kernel/read
dense_2/Square
dense_2/mul/x
dense_2/mul
dense_2/Const
dense_2/Sum
dense_2/add/x
dense_2/add
dense_2/Const_1
dense_2/bias
dense_2/bias/Assign
dense_2/bias/read
dense_2

In [21]:
#holdout_summaries = [
#        graph_dense.get_tensor_by_name('evaluation/holdout_loss:0'),
#        graph_dense.get_tensor_by_name('evaluation/holdout_accuracy:0')
#    ]

hol = graph_dense.get_tensor_by_name('evaluation/holdout_loss:0')
sums = graph_dense.get_tensor_by_name('evaluation/holdout_summaries:0')


#sums = graph_dense.get_operation_by_name('evaluation/Merge/MergeSummary')
#batch_summaries = [
#        graph_dense.get_operation_by_name('evaluation/batch_loss'),
#        graph_dense.get_operation_by_name('evaluation/batch_accuracy')
#    ]

with tf.Session(graph=graph_dense) as sess:
    K.set_session(sess)
    sess.run(init_op)
        
    x, y = sess.run([hol, sums], feed_dict={board: ds_test.X_test, outcome: ds_test.y_test})
    #z = graph_dense.get_operation_by_name('evaluation/holdout_loss').run(feed_dict={board: ds_test.X_test, outcome: ds_test.y_test})
    
    print x, y



evaluation/holdout_loss�̌? 

evaluation/holdout_loss�̌?
"
evaluation/holdout_accuracy��>


In [15]:
x

[None]

## Arch 2: Pure CovNet

In [ ]:
# Create a graph to hold the model.
graph_cov_pure = tf.Graph()

# Create model in the graph.
with graph_cov_pure.as_default():
    
    # Keras layers can be called on TensorFlow tensors:
    board = tf.placeholder(tf.float32, shape=(None, 42), name='board') 
    outcome = tf.placeholder(tf.float32, shape=(None, 3), name='outcome')    
    
    # The input data is [col0=[row_0, row_1, ...], col1=[row_0, row_1], ...]
    rs = Reshape((7, 6, 1), input_shape=(42,))(board)
    
    conv_args = dict(
        use_bias=True,
        activation='relu',
        kernel_initializer='random_uniform',
        bias_initializer='zeros',
        kernel_regularizer=regularizers.l2(0.01),
        input_shape=(7, 6, 1),
        padding='valid'
    )
    
    # We use a few parallel covents, that we combine in the end        
    c1 = (Conv2D(8,  kernel_size=(1, 2), **conv_args)(rs))
    c2 = (Conv2D(16, kernel_size=(2, 1), **conv_args)(c1))
    c3 = (Conv2D(32, kernel_size=(3, 3), **conv_args)(c2))
    #c4 = (Conv2D(64, kernel_size=(6, 6), **conv_args)(c3))
    
    dense_args = dict(
        use_bias=True,
        activation='relu',
        kernel_initializer='random_uniform',
        bias_initializer='zeros',
        kernel_regularizer=regularizers.l2(0.01),
        bias_regularizer=regularizers.l2(0.01),    
    )
    
    d = Dense(64,  **dense_args)(Flatten()(c3))         
    
    # output layer with 10 units and a softmax activation
    preds = Dense(3, activation='softmax', name='preds')(d) 
    
    with tf.name_scope('evaluation') as scope:
        loss = tf.reduce_mean(categorical_crossentropy(outcome, preds), name='loss')
        tf.summary.scalar('holdout_loss', loss)
        
        acc_value = tf.identity(accuracy(outcome, preds), name='accuracy')
        tf.summary.scalar('holdout_accuracy', tf.reduce_mean(acc_value))
    
    with tf.name_scope('training') as scope:
        train_step = tf.train.AdamOptimizer(learning_rate=0.01).minimize(loss, name='train_step')    
    
    # Initialize all variables
    init_op = tf.global_variables_initializer()
    
    all_summaries = tf.summary.merge_all()

## Arch 3: Complicated CovNet

In [ ]:
# Create a graph to hold the model.
graph_cov_comp = tf.Graph()

# Create model in the graph.
with graph_cov_comp.as_default():
    
    # Keras layers can be called on TensorFlow tensors:
    board = tf.placeholder(tf.float32, shape=(None, 42), name='board') 
    outcome = tf.placeholder(tf.float32, shape=(None, 3), name='outcome')    
    
    # The input data is [col0=[row_0, row_1, ...], col1=[row_0, row_1], ...]
    rs = Reshape((7, 6, 1), input_shape=(42,))(board)
    
    conv_args = dict(
        use_bias=True,
        activation='relu',
        kernel_initializer='random_uniform',
        bias_initializer='zeros',
        kernel_regularizer=regularizers.l1_l2(1.0),
        bias_regularizer=regularizers.l1_l2(1.0),
#        input_shape=(7, 6, 1),
        padding='valid'
    )
    
    # We use a few parallel covents, that we combine in the end        
    ca = Flatten()(Conv2D(12, kernel_size=(5, 5), **conv_args)(rs))
    cb = Flatten()(Conv2D(12, kernel_size=(4, 4), **conv_args)(rs))
    cc = Flatten()(Conv2D(12, kernel_size=(3, 3), **conv_args)(rs))

    cg = Flatten()(Conv2D(8, kernel_size=(6, 1), **conv_args)(rs))
    ch = Flatten()(Conv2D(8, kernel_size=(1, 6), **conv_args)(rs))
    
    cd = Flatten()(Conv2D(4, kernel_size=(1, 4), **conv_args)(rs))
    ce = Flatten()(Conv2D(4, kernel_size=(4, 1), **conv_args)(rs))
    cf = Flatten()(Conv2D(4, kernel_size=(2, 2), **conv_args)(rs))

    dense_args = dict(
        use_bias=True,
        activation='relu',
        kernel_initializer='random_uniform',
        bias_initializer='zeros',
        kernel_regularizer=regularizers.l1_l2(1.0),
        bias_regularizer=regularizers.l1_l2(1.0),    
    )
    
    d1 = Dense(512, **dense_args)(board)
    
    merged = Concatenate()([d1, ca, cb, cc, cd, ce, cf, cg, ch])    
        
    x = Dense(512, **dense_args)(merged)         
    x = Dense(256, **dense_args)(x)         
    x = Dense(128, **dense_args)(x)    
    x = Dense(12,  **dense_args)(x)
    
    # output layer with 10 units and a softmax activation
    preds = Dense(3, activation='softmax', name='preds')(x) 
    
    with tf.name_scope('evaluation') as scope:
        loss = tf.reduce_mean(categorical_crossentropy(outcome, preds), name='loss')
        tf.summary.scalar('holdout_loss', loss)
        
        acc = tf.identity(accuracy(outcome, preds), name='accuracy')
        tf.summary.scalar('holdout_accuracy', tf.reduce_mean(acc))
    
    with tf.name_scope('training') as scope:
        train_step = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(loss, name='train_step')    
    
    # Initialize all variables
    init_op = tf.global_variables_initializer()
    
    all_summaries = tf.summary.merge_all()

-----------------------

# Gen 1

In [ ]:
#simulations/random-2017-10-28-17:13:04

# 'random-2017-10-21-13:41:47'
# 'random-2017-10-28-17:13:04'

ds_gen1 = (training.DataLoader(frac_train=0.95, frac_test=0.05)
           .add_dataset('random-2017-10-28-17:13:04')
           .load())

### Fully Connected

In [ ]:
training.train(graph_dense, './models/dense_{}'.format(datetime.datetime.now().strftime("%Y_%m_%d_%H%M%S")),
               ds_gen1,  batch_size=500, num_batches=10000)

### Covnet

In [ ]:
training.train(graph_cov_pure, './models/cov_pure_{}'.format(datetime.datetime.now().strftime("%Y_%m_%d_%H%M%S")),
               ds_gen1,  batch_size=200, num_batches=20000)


### Advanced Covnet

In [ ]:
# Best: gen1-cov2d_beta_2017_10_29_150829
# Dataset: 'random-2017-10-28-17:13:04'
# batch_size=500, learning_rate=0.001, regularization=(l1_l2, 1.0) (regularization of 0.1 seems to have the same effect...)
# Include all convolutions (adding cc, cf, cg, ch)
# Include 4 layers of dense: 512, 256, 128, 12

training.train(graph_cov_comp, './models/gen1-cov2d_alpha_{}'.format(datetime.datetime.now().strftime("%Y_%m_%d_%H%M%S")),
               ds_gen1, batch_size=500, num_batches=15000)

-----------------

# Gen 2

In [ ]:
ds_gen2 = (training.DataLoader(frac_train=0.95, frac_test=0.05)
           .add_dataset('random-2017-10-28-17:13:04', 100000)
           .add_dataset('gen1-cov2d_beta_2017_10_29_150829-2017-10-29-16:57:41')
           .load())

# Use all the advanced data
# 10,000 rows of gen-1 vs gen-1 data
#key = 'gen-1-cov2d_beta_2017_10_22_142925'
#features, targets, features_train, target_train, features_test, target_test = load_data('training_data/gen-1-cov2d_beta_2017_10_22_142925')

In [ ]:
training.train(graph_cov_comp, './models/gen2-cov2d_beta_{}'.format(datetime.datetime.now().strftime("%Y_%m_%d_%H%M%S")),
               ds_gen2, batch_size=250, num_batches=20000)